# Lab 13:  K-Nearest Neighbors Regression

The *k-nearest neighbors* algorithm makes its predictions using the values of the k closest training data points.  For example, the 3-nearest neighbor algorithm will find the 3 closest data points (using the Euclidean distance) in the training data and use them to make a prediction.

If we are performing regression (trying to predict a quantitative value), the prediction is the mean of the y values of the k neighbors.

For this lab, we will use the building sale price data from Homework 9, which contains information about building sales in New York City from September 2016 to September 2017.  We will used the somewhat cleaned [version from Kaggle](https://www.kaggle.com/new-york-city/nyc-property-sales), which was originally taken from [New York City's Department of Finance Rolling Sales dataset](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page).

Data URL:  [https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/nyc-rolling-sales.csv](https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/nyc-rolling-sales.csv)

### Section 1: Loading and cleaning the data

First import the libraries we will use.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import MinMaxScaler
    
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

%matplotlib inline

pd.set_option('display.max_columns', None)

Load the data into the DataFrame `nyc`.  Missing values are stored as " -  " (a space, a a dash, and then two spaces) in the CSV file, so include the parameter to have Pandas convert them to Nan.

<details><summary>Hint:</summary>
    Use the parameter `na_values = [" - "]` to have Pandas store the missing data as Nan.
    </details>
    
Display your DataFrame if you haven't already.

In this lab, we will try to predict the sales price from other information about the building.  Look at the columns - which do you think would not be helpful in determining the sales price or can not be easily used as/converted into quantitative data?  We will drop such columns.

Qualitative columns will need to be made into dummy variables, which could add a lot of new columns (ex. for the neighborhood column) and slow down computation speed.  So we are also balancing how many new columns will be added with how useful we think that information will be.  

Drop the following columns:
* "Unnamed: 0" (some kind of left-over index column?)
* "EASE-MENT" (seems empty for a lot of rows)
* "ADDRESS"  (could be helpful for determining the sales price, but we would have to do something further with it, such as extracting the street name)
* "APARTMENT NUMBER"  (empty for a lot of rows)
* "BLOCK" (could be helpful, but likely there are very few repeated values)
* "LOT" (lot numbers will be the same in different blocks, but that doesn't tell us anything about the property value)
* "TAX CLASS AT PRESENT" (we will keep the "TAX CLASS AT TIME OF SALE" instead, but an argument could be made to keep "TAX CLASS AT PRESENT" instead)
* "BUILDING CLASS AT PRESENT"  (we will keep the "TAX CLASS AT TIME OF SALE" instead, but an argument could be made to keep "BUILDING CLASS AT PRESENT" instead)


<details><summary>Hint:</summary>
Pattern for dropping multiple columns:
    <code>df.drop(columns = ["col1", "col2", "col3"], inplace = True)</code>
</details>

Next let's drop any rows with missing data.

Check the length of the new DataFrame to make sure we still have enough data.

Because both the sale price and gross square feet column have outliers (feel free to plot each as a histogram to verify), we will filter the data as in Homework 9 to remove these outliers.

First filter the DataFrame to only include sale prices between 20,000 and 2,000,000:

Next filter the DataFrame to only include gross square feet greater than 0 or less than 6000 sq. ft.:

The sale date might affect the sale price, but right now the `SALE DATE` column is not in a format that can be used to make predictions.  To change it to a usable format, we should:
* change the type of the column to a DateTime object
* create a new column containing only the month
* create a new column containing only the yera
* drop the `SALE DATE` column (as a DateTime type), since it still can't be used by scikit-learn

Finally, make most of the qualitative columns into dummy variables.  We will treat `MONTH` as a quantitative variable, under the assumption that sales in consecutive months are similar (though this will not cover the December-January).

Specifically, make the following columns into dummy variables: `"BOROUGH","NEIGHBORHOOD","BUILDING CLASS CATEGORY", "ZIP CODE", "TAX CLASS AT TIME OF SALE", "BUILDING CLASS AT TIME OF SALE"`

How many columns does our new dataset have? 



 It is ok that it is so large!  Part of what makes data science so powerful is that modern computers can quickly do computations even when we have a lot of variables.

### Section 2: K-Nearest Neighbors

First create a new DataFrame `x` with only the independent variables (every column except `SALE PRICE`).

Also create a new Series `y` with only the dependent variable (`SALE PRICE`).

Split the `x` and `y` data into a training and testing set using `train_test_split()`.  Use a proportion of 0.2 of the data for test.

The following code creates a 7-nearest neighbor regressor object and fits it to our training data.

In [ ]:
knn = KNeighborsRegressor(n_neighbors = 7)
knn.fit(x_train, y_train)

We can then use it to make predictions from our test data:

In [ ]:
y_test_pred = knn.predict(x_test)

Compute the mean squared error of the test predictions.

<details><summary>Answer:</summary>
<code>mean_squared_error(y_test_pred, y_test)</code>
</details>

We'll compare this mean squared error (MSE) to the MSE of the training data to check for overfitting.

First predict `y` from the training data, and then compute its mean squared error.

How do the two mean squared error compare?  Do you think the model overfits the data?

Just as with linear regression, we can look for systematic error (error that has a pattern) by plotting the true `y` values of the test data on the x-axis against the error (true `y` values - predicted `y` values) on the y-axis.  Can you figure out the code to do this?

<details><summary>Answer:</summary>
<code>plt.scatter(y_test, y_test - y_test_pred, alpha = 0.5)</code>
</details>

Which prices are being under-estimated (the error is positive) by our model?  Which prices are being over-estimated (the error is negative) by our model?

### Section 3: Scaling

Notice that the values in some columns, like `LAND SQUARE FEET` and `GROSS SQUARE FEET`, are large, while the values in all the dummy variables columns are small (0 or 1).

When the columns have different scales, the largest column(s) will dominate.  We can get better results by scaling all of our columns to be between 0 and 1.  The scaling formula is:

$$x_{scaled} = \frac{x - x_{\min}}{x_{\max} - x_{\min}}$$

We can use a built in function in sci-kit learn to do the scaling.  First we make a scaler object.

In [ ]:
scaler = MinMaxScaler()

And then we scale the training data.

In [ ]:
x_train_scaled = scaler.fit_transform(x_train)

What does the scaled data look like?  Is this what you expect?

Scale your x test data.  We do not need to scale the y data.

Create and fit a 7-nearest neighbor regressor with the scaled training data.

Use this new model to make predictions from the scaled test data.

Compute the mean squared error for these predictions.  Does scaling improve the 3-7earest neighbor regressor?

### Section 4:  Parameter search

In k-nearest neighbors, k is called a *parameter* (similar to the parameters we pass into our code functions).  We can try to figure out the best k to use by trial and error, but it might be easier to systematically try all values of k in some range.

Thus, we will write a loop to try all values of k between 1 and 20, and compute the mean squared error for each one.  The pseudo-code to do this is:

<code>
create an empty list
loop k from 1 to 20:
    create a k-nearest neighbor regressor
    fit the training data to the k-nearest neighbor regressor
    make predictions for the test data
    compute the mean squared error for the predictions
    store the mean squared error in the list
</code>

The code will take some time to run.  It will take longer to run for large k, so you can reduce the time by searching a smaller range of possible k's.

In [ ]:
mses = []
for k in range(1,21):
    print("Now computing MSE for k=",k)
    iknn_scaled = KNeighborsRegressor(n_neighbors = k)
    iknn_scaled.fit(x_train_scaled, y_train)
    iy_pred_scaled = iknn_scaled.predict(x_test_scaled)
    mse = mean_squared_error(iy_pred_scaled, y_test)
    mses.append(mse)

We can plot these mean squared errors as a line plot, to understand how increasing k changes the MSE.

In [ ]:
plt.plot(mses)
plt.xlabel("k")
plt.ylabel("MSE")
plt.title("MSE for different numbers of neighbors")

How do the mean squared errors change?  What k would you choose?  What other information would help you make this choice?